In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns

# Load dataset
file_path = '/content/bangladesh_satellite_weather_data.xlsx'  # Adjust if needed
data = pd.read_excel(file_path)

# Define input features and target variable
X = data[['Longitude', 'Latitude', 'Elevation', 'Antenna Gain', 'Transmitter Power',
          'Uplink Frequency', 'Polarization Type', 'Beam Type', 'Polarization Match',
          'Transmission Mode', 'Satellite Latitude', 'Satellite Longitude', 'Satellite Antenna Gain',
          'Rain Rate', 'Surface Temperature', 'Total Surface Pressure',
          'Humidity Level', 'Cloud Thickness', 'Liquid Water Density in Clouds', 'Liquid Water Temperature in Clouds', 'Path Length']]

y = data['Total Attenuation']  # Target variable

# One-Hot Encoding for categorical columns
categorical_columns = ['Polarization Type', 'Beam Type', 'Polarization Match', 'Transmission Mode']
X_encoded = pd.get_dummies(X, columns=categorical_columns)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### 🔹 Step 1: Train Initial Random Forest Model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train_scaled, y_train)
y_pred_rf = rf.predict(X_test_scaled)

# Evaluate Initial Random Forest model
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print(f"Initial Random Forest MSE: {mse_rf}, R-squared: {r2_rf}")

### 🔹 Step 2: Optimize Random Forest with `RandomizedSearchCV`
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt'],
    'bootstrap': [True, False]
}

random_search = RandomizedSearchCV(RandomForestRegressor(random_state=42), param_distributions=param_grid,
                                   n_iter=10, scoring='r2', cv=3, n_jobs=-1, random_state=42)
random_search.fit(X_train_scaled, y_train)

best_rf = random_search.best_estimator_
y_pred_best_rf = best_rf.predict(X_test_scaled)

# Evaluate Optimized Random Forest model
mse_best_rf = mean_squared_error(y_test, y_pred_best_rf)
r2_best_rf = r2_score(y_test, y_pred_best_rf)

print(f"Optimized Random Forest MSE: {mse_best_rf}, R-squared: {r2_best_rf}")

### 🔹 Step 3: Feature Importance Analysis
best_rf.fit(X_train_scaled, y_train)
feature_importances = best_rf.feature_importances_

plt.figure(figsize=(10, 5))
sns.barplot(x=feature_importances, y=X_encoded.columns)
plt.xlabel("Feature Importance")
plt.ylabel("Feature Name")
plt.title("Random Forest Feature Importance")
plt.show()

### 🔹 Step 4: Plot Results
plt.figure(figsize=(7, 5))
plt.scatter(y_test, y_pred_best_rf, label='Random Forest Prediction', alpha=0.6)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red')
plt.xlabel('Actual Path Loss (dB)')
plt.ylabel('Predicted Path Loss (dB)')
plt.title('Random Forest: Actual vs Predicted Path Loss')
plt.legend()
plt.show()


# Predict from manual input
# Example input row
#input_row =[91.9963, 22.5475, 49.715, 54.5,  2200, 4.544461186, 'Vertical', 'Narrow', 'Vertical', 'Continuous', 0, 119.1, 111.2, 4.89, 18.06, 1014.7, 43.79, 1.12, 0.22, 17.19, 35786]
#input_row= [90.3846, 23.9930, 49.715, 61.2, 1250, 12.01881235, 'Horizontal', 'Wide', 'Horizontal', 'Continuous', 0, 119.1, 125.4, 34.53, 28.37, 1006.39, 89.5, 3.35, 0.18, 25.79, 35786]


print("Please enter the following values:")

longitude = float(input("Longitude: "))
latitude = float(input("Latitude: "))
elevation = float(input("Elevation: "))
antenna_gain = float(input("Antenna Gain: "))
transmitter_power = float(input("Transmitter Power: "))
uplink_frequency = float(input("Uplink Frequency: "))
polarization_type = input("Polarization Type (Vertical/Horizontal): ")
beam_type = input("Beam Type (Narrow/Wide): ")
polarization_match = input("Polarization Match (Vertical/Horizontal): ")
transmission_mode = input("Transmission Mode (Continuous/Intermittent): ")
satellite_latitude = float(input("Satellite Latitude: "))
satellite_longitude = float(input("Satellite Longitude: "))
satellite_antenna_gain = float(input("Satellite Antenna Gain: "))
rain_rate = float(input("Rain Rate: "))
surface_temperature = float(input("Surface Temperature: "))
total_surface_pressure = float(input("Total Surface Pressure: "))
humidity_level = float(input("Humidity Level: "))
cloud_thickness = float(input("Cloud Thickness: "))
liquid_water_density = float(input("Liquid Water Density in Clouds: "))
liquid_water_temperature = float(input("Liquid Water Temperature in Clouds: "))
path_length = float(input("Path Length: "))

input_row = [
    longitude, latitude, elevation, antenna_gain, transmitter_power,
    uplink_frequency, polarization_type, beam_type, polarization_match,
    transmission_mode, satellite_latitude, satellite_longitude, satellite_antenna_gain,
    rain_rate, surface_temperature, total_surface_pressure, humidity_level,
    cloud_thickness, liquid_water_density, liquid_water_temperature, path_length
]

# Convert input row to DataFrame (Now correctly formatted)
input_df = pd.DataFrame([input_row], columns=X.columns)

# One-Hot Encode categorical variables
input_df_encoded = pd.get_dummies(input_df, columns=categorical_columns)

# Ensure all required columns exist
missing_cols = set(X_encoded.columns) - set(input_df_encoded.columns)
for col in missing_cols:
    input_df_encoded[col] = 0  # Assign 0 to missing categorical features

# Reorder columns to match training data
input_df_encoded = input_df_encoded[X_encoded.columns]

# Scale the input
input_scaled = scaler.transform(input_df_encoded)

# Predict Free Space Path Loss
predicted_loss = best_rf.predict(input_scaled)
print(f"Predicted Total Attenuation: {predicted_loss[0]} dB")



ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-b12f5cb9a380>", line 12, in <cell line: 0>
    data = pd.read_excel(file_path)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/io/excel/_base.py", line 508, in read_excel
    data = io.parse(
           ^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/io/excel/_base.py", line 1616, in parse
    return self._reader.parse(
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/io/excel/_base.py", line 778, in parse
    data = self.get_sheet_data(sheet, file_rows_needed)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/io/excel/_openpyxl.py", line 615, in get_sheet_data
    for row_number, row in enumerate(sheet.rows):
  Fi